# Real-World Data Wrangling

## 1. Gather data

In this section, we will extract data using at least two different data gathering methods and combine the data.

### **1.1.** Problem Statement

How does the streaming revolution compare to traditional theatrical releases in terms of financial performance, audience reception, and content characteristics, and how have these metrics evolved over time when accounting for inflation?

The film industry has undergone a dramatic shift from theatrical dominance to streaming platforms, but how do these distribution channels compare in terms of content characteristics, audience engagement, and financial viability?

This project merges TMDb theatrical movie data, IMDb ratings, four major streaming platform catalogs, and Federal Reserve CPI data to conduct a comprehensive analysis of streaming-exclusive versus theatrical releases. By adjusting all budgets and revenues to 2024 dollars using inflation data, I can fairly compare financial performance across six decades (1960-2024) and determine whether the "streaming revolution" represents truly different content or simply a new distribution channel for similar movies.

The primary data wrangling challenge involves fuzzy matching movies across datasets with inconsistent identifiers, requiring normalized title matching and year-based validation.

### **1.2.** Dataset Overview

This project combines four distinct data sources gathered using three different methods:

| Dataset | Source | Method | Size | Primary Use |
|---------|--------|--------|------|-------------|
| TMDb Movies | Kaggle | Manual Download | 10K movies | Theatrical releases, financial data |
| IMDb Ratings | IMDb Datasets | Programmatic Download | ~50K movies (filtered) | Additional ratings, metadata validation |
| Streaming Platforms | Kaggle | Manual Download | ~22K titles | Streaming availability, exclusive content |
| CPI Inflation Data | FRED API | API Access | 65 years (1960-2024) | Inflation adjustment for financial analysis |

The combination of these datasets enables analysis of streaming versus theatrical releases while ensuring fair financial comparisons across six decades.

In [1]:
# Import statements for packages used in analysis
import httpx
import os
import zlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sb

from dotenv import load_dotenv
from urllib.parse import urlparse

## It's OK if this file doesn't exist as the API results are 
## saved to a file in the project and the API code will check
## for the file before using the API key this would retrieve.
load_dotenv(dotenv_path=f"{os.path.expanduser('~/.env')}")

# Magic word for inline visualizations
%matplotlib inline

# Set default figure size for better readability
sb.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

In [2]:
## Data gathering and display functions
def download_and_save_file(url, local_filename, path="data/raw", download_if_exists=False):
    """
    Downloads a file from a URL and saves it locally.
    By default the file is saved to the data/raw path.
    
    If the file is gzipped, it's decompressed on-the-fly and
    written as uncompressed data.

    Args:
        url (str): URL of file to download.
        local_filename (str): Name of file to save locally.
    """
    filepath = f"{path}/{local_filename}"
    
    if os.path.exists(filepath) and download_if_exists is False:
        print(f"{filepath} already exists. If you wish to re-download it, set download_if_exists=True")
        return
    
    try:
        with httpx.stream("GET", url, headers={"Accept-Encoding": "gzip"}) as response:
            # Raise an exception for bad status codes
            response.raise_for_status()

            headers = response.headers
            is_gzip = False
            
            print(f"Downloading {url} --> {filepath}")

            # Check if response is gzipped
            if 'content-encoding' in headers and 'gzip' in headers['content-encoding']:
                    is_gzip = True

            elif 'content-type' in headers and 'binary/octet-stream' in headers['content-type']:                
                # AWS likes to send this back; just check the extension
                _, extension = os.path.splitext(urlparse(url).path)

                if extension == '.gz':
                    is_gzip = True

            # Decompress file if it's gzipped
            if is_gzip is True:
                print("Decompressing gzip content...")

                decompressor = zlib.decompressobj(wbits=zlib.MAX_WBITS | 16)
                download_size = 0
                count = 0

                with open(filepath, "wb") as outfile:
                    for chunk in response.iter_bytes():
                        count += 1
                        download_size += len(chunk)
                        decompressed_chunk = decompressor.decompress(chunk)

                        if download_size % 1000000 == 0:
                            print(f"Downloaded bytes: {download_size:,}")

                        outfile.write(decompressed_chunk)

                print(f"Downloaded {count:,} chunks for file size {download_size:,}")
                
                remaining_data = decompressor.flush()
                if remaining_data:
                    with open(filepath, "wb") as outfile:
                        outfile.write(remaining_data)


            # If not gzipped, just write raw content to the output file
            else:
                with open(filepath, 'wb') as outfile:
                    for chunk in response.iter_bytes():
                        outfile.write(chunk)

            # Give the user feedback
            print(f"File '{local_filename}' downloaded successfully from '{url}'")

    # Check for various issues
    except httpx.HTTPStatusError as e:
        print(f"HTTP error occurred: {e}")
        
    except httpx.RequestError as e:
        print(f"An error occurred while requesting {url}: {e}")
        
    except IOError as e:
        print(f"Error saving file '{local_filename}': {e}")

def display_shape(df):
    """
    Output rows and columns of dataframe
    """
    print(f"\nRows: {df.shape[0]:,}")
    print(f"Columns: {df.shape[1]:,}\n")

#### TMDb Movies Dataset

The TMDb (The Movie Database) dataset provides comprehensive information about 10,000 theatrical movies, representing the foundation of this analysis. This dataset is particularly valuable because it includes both original and inflation-adjusted financial figures (budget_adj and revenue_adj to 2010 dollars), extensive production metadata, and audience reception metrics. TMDb's community-driven model ensures broad coverage of theatrical releases while maintaining high data quality through active curation.

Source URL: https://www.kaggle.com/datasets/juzershakir/tmdb-movies-dataset

Type: CSV (6.85 MB)

Method: Manual download from Kaggle

**Dataset variables:**
- id (int) - TMDb unique identifier
- imdb_id (string) - IMDb identifier for cross-dataset linking
- popularity (float) - TMDb popularity score
- budget (int) - Production budget in nominal dollars
- revenue (int) - Total box office revenue in nominal dollars
- original_title (string) - Original title
- cast (string) - Cast members (pipe-delimited)
- homepage (string) - Official movie homepage URL
- director (string) - Director name
- tagline (string) - Marketing catchphrase
- keywords (string) - Descriptive keywords (pipe-delimited)
- overview (string) - Plot synopsis
- runtime (int) - Duration in minutes
- genres (string) - Movie genres (pipe-delimited)
- production_companies (string) - Production companies (pipe-delimited)
- release_date (string) - Release date (format: 6/9/2015)
- vote_count (int) - Total number of TMDb user votes
- vote_average (float) - Average TMDb user rating (0-10 scale)
- release_year (int) - Release year
- budget_adj (float) - Budget adjusted to 2010 dollars
- revenue_adj (float) - Revenue adjusted to 2010 dollars

**Why this dataset:**
TMDb serves as the core theatrical release dataset because it combines financial data (budget/revenue), audience reception (ratings/votes), production details (cast/director/companies), and content characteristics (genres/runtime) in a single source. The pre-existing inflation adjustment to 2010 demonstrates awareness of temporal comparison challenges, though this project will re-adjust all figures to 2024 dollars for consistency. The imdb_id field enables precise linking to IMDb data, while the comprehensive metadata supports fuzzy matching to streaming platforms where IDs are absent.

**Significance of variables:**
- **id/imdb_id:** Primary keys for merging with IMDb and tracking across datasets
- **budget/revenue (and _adj versions):** Core financial metrics for profitability analysis; will be re-adjusted to 2024 dollars
- **vote_average/vote_count:** TMDb's rating system for comparison with IMDb's averageRating/numVotes
- **genres/runtime:** Content characteristics for analyzing streaming vs theatrical patterns
- **cast/director:** Enable talent crossover analysis between distribution channels
- **release_year/release_date:** Temporal analysis and fuzzy matching keys
- **popularity:** TMDb-specific engagement metric not available in other datasets

In [3]:
# Load dataset
tmdb_movies_raw_df = pd.read_csv('data/raw/tmdb_movies_data.csv')

display_shape(tmdb_movies_raw_df)
display(tmdb_movies_raw_df.head())


Rows: 10,866
Columns: 21



,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/15,5562,6.5,2015,1.379999e+08,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/15,6185,7.1,2015,1.379999e+08,3.481613e+08
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/15,2480,6.3,2015,1.012000e+08,2.716190e+08
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/15,5292,7.5,2015,1.839999e+08,1.902723e+09
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/15,2947,7.3,2015,1.747999e+08,1.385749e+09


#### IMDb Non-Commercial Datasets

IMDb provides free access to subsets of their data for non-commercial use, refreshed daily. This dataset extends the TMDb data with additional ratings and metadata, and provides validation for fuzzy matching when IMDb IDs are missing. IMDb's community-driven ratings represent a different audience demographic than TMDb, enabling comparison of rating systems.

Source URL: https://datasets.imdbws.com/

Type: TSV (Tab-Separated Values), gzip-compressed

Method: Programmatic download using Python httpx library

**Files Used:**

**title.basics.tsv.gz** - Core movie metadata
   - tconst (string) - IMDb unique identifier (e.g., tt0111161)
   - titleType (string) - Format type (movie, short, tvSeries, etc.)
   - primaryTitle (string) - Popular title used in marketing
   - originalTitle (string) - Original language title
   - isAdult (boolean) - Adult content flag (0 or 1)
   - startYear (YYYY) - Release year
   - runtimeMinutes (int) - Duration in minutes
   - genres (string array) - Up to three genres (comma-separated)

**title.ratings.tsv.gz** - Community ratings
   - tconst (string) - IMDb unique identifier
   - averageRating (float) - Weighted average rating (1-10 scale)
   - numVotes (int) - Total number of votes

**Why this dataset:**
IMDb's rating system is well-established and trusted, providing an alternative measure of audience reception. The tconst identifier enables precise linking to TMDb via imdb_id field, while the comprehensive coverage helps identify theatrical releases that may be missing from TMDb. The daily-updated nature ensures current data.

**Significance of variables:**
- **tconst/averageRating/numVotes:** Enable comparison of IMDb vs TMDb rating systems and vote engagement
- **titleType:** Filters dataset to movies only (excluding TV series, shorts)
- **startYear/runtimeMinutes:** Provide additional data points for movies missing this info in TMDb
- **genres:** Cross-validation of genre classifications across systems

>**title.basics.tsv.gz** - Core movie metadata

In [4]:
# Get files via URL download
URL = "https://datasets.imdbws.com/title.basics.tsv.gz"
LOCAL_FILE = "imdb_title_basics.tsv"

download_and_save_file(URL, LOCAL_FILE)

# Load dataset - note that this dataset is quite large at almost 1GB on disk
imdb_title_basics_raw_df = pd.read_csv(f'data/raw/{LOCAL_FILE}', sep='\t')

display_shape(imdb_title_basics_raw_df)
display(imdb_title_basics_raw_df.head())

data/raw/imdb_title_basics.tsv already exists. If you wish to re-download it, set download_if_exists=True

Rows: 12,116,115
Columns: 9



,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,Short


>**title.ratings.tsv.gz** - Community ratings

In [5]:
# Get files via URL download
URL = "https://datasets.imdbws.com/title.ratings.tsv.gz"
LOCAL_FILE="imdb_title_ratings.tsv"

download_and_save_file(URL, LOCAL_FILE)

# Load dataset
imdb_title_ratings_raw_df = pd.read_csv(f'data/raw/{LOCAL_FILE}', sep='\t')

display_shape(imdb_title_ratings_raw_df)
display(imdb_title_ratings_raw_df.head())

data/raw/imdb_title_ratings.tsv already exists. If you wish to re-download it, set download_if_exists=True

Rows: 1,607,373
Columns: 3



,tconst,averageRating,numVotes
0,tt0000001,5.7,2187
1,tt0000002,5.5,307
2,tt0000003,6.4,2273
3,tt0000004,5.2,196
4,tt0000005,6.2,3012


#### Streaming Platform Catalogs

Four major streaming platform datasets provide comprehensive coverage of content available on Netflix, Hulu, Disney+, and Amazon Prime as of mid-2021. These datasets enable identification of streaming-exclusive content (never released theatrically) versus theatrical films later added to streaming platforms. The consistent schema across platforms facilitates comparison of content strategies.

**Datasets:**
- Netflix: https://www.kaggle.com/datasets/shivamb/netflix-shows (~8K titles)
- Hulu: https://www.kaggle.com/datasets/shivamb/hulu-movies-and-tv-shows (~3K titles)  
- Disney+: https://www.kaggle.com/datasets/shivamb/disney-movies-and-tv-shows (~1.3K titles)
- Amazon Prime: https://www.kaggle.com/datasets/shivamb/amazon-prime-movies-and-tv-shows (~10K titles)

Type: CSV

Method: Manual download from Kaggle (all four datasets)

**Shared Schema:**
- show_id (string) - Unique identifier per platform
- type (string) - "Movie" or "TV Show"
- title (string) - Content title
- director (string) - Director name(s)
- cast (string) - Cast members (comma-separated)
- country (string) - Production country
- date_added (string) - Date added to platform (e.g., "September 25, 2021")
- release_year (int) - Original release year
- rating (string) - Content rating (PG-13, TV-MA, etc.)
- duration (string) - Runtime (e.g., "90 min" or "2 Seasons")
- listed_in (string) - Genres (comma-separated)
- description (string) - Synopsis

**Why these datasets:**
Streaming platforms represent the industry's future, but comprehensive availability data isn't accessible via APIs without commercial partnerships. These Kaggle datasets provide a snapshot of four major platforms' catalogs at a single point in time (2021), enabling analysis of platform content strategies. The lack of consistent identifiers (no TMDb/IMDb IDs) creates an authentic data wrangling challenge requiring fuzzy matching.

**Significance of variables:**
- **type:** Filters to movies only (excluding TV shows)
- **title/release_year:** Primary keys for fuzzy matching to theatrical databases
- **director/cast:** Enable cross-validation of matches and analysis of talent migration to streaming
- **date_added:** Indicates when content became available (may differ from release_year)
- **listed_in:** Genre analysis comparing streaming vs theatrical content preferences
- **duration:** Identifies potential data quality issues (TV shows mis-labeled as movies)

>**Amazon Prime**

In [6]:
# I decided to manually load these datasets into the project.
# Working with the kaggle and kagglehub Python packages is more trouble than it's worth.

amazon_prime_titles_raw_df = pd.read_csv('data/raw/amazon_prime_titles.csv')
display_shape(amazon_prime_titles_raw_df)
display(amazon_prime_titles_raw_df.head())


Rows: 9,668
Columns: 12



,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ..."
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...


>**Disney Plus**

In [7]:
disney_plus_titles_raw_df = pd.read_csv('data/raw/disney_plus_titles.csv')
display_shape(disney_plus_titles_raw_df)
display(disney_plus_titles_raw_df.head())


Rows: 1,450
Columns: 12



,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!
1,s2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",NaN,"November 26, 2021",1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...
2,s3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,"November 26, 2021",2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.
3,s4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",NaN,"November 26, 2021",2021,TV-PG,41 min,Musical,"This is real life, not just fantasy!"
4,s5,TV Show,The Beatles: Get Back,NaN,"John Lennon, Paul McCartney, George Harrison, ...",NaN,"November 25, 2021",2021,NaN,1 Season,"Docuseries, Historical, Music",A three-part documentary from Peter Jackson ca...


>**Hulu**

In [8]:
hulu_titles_raw_df = pd.read_csv('data/raw/hulu_titles.csv')
display_shape(hulu_titles_raw_df)
display(hulu_titles_raw_df.head())


Rows: 3,073
Columns: 12



,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Ricky Velez: Here's Everything,NaN,NaN,NaN,"October 24, 2021",2021,TV-MA,NaN,"Comedy, Stand Up",​Comedian Ricky Velez bares it all with his ho...
1,s2,Movie,Silent Night,NaN,NaN,NaN,"October 23, 2021",2020,NaN,94 min,"Crime, Drama, Thriller","Mark, a low end South London hitman recently r..."
2,s3,Movie,The Marksman,NaN,NaN,NaN,"October 23, 2021",2021,PG-13,108 min,"Action, Thriller",A hardened Arizona rancher tries to protect an...
3,s4,Movie,Gaia,NaN,NaN,NaN,"October 22, 2021",2021,R,97 min,Horror,A forest ranger and two survivalists with a cu...
4,s5,Movie,Settlers,NaN,NaN,NaN,"October 22, 2021",2021,NaN,104 min,"Science Fiction, Thriller",Mankind's earliest settlers on the Martian fro...


>**Netflix**

In [9]:
netflix_titles_raw_df = pd.read_csv('data/raw/netflix_titles.csv')
display_shape(netflix_titles_raw_df)
display(netflix_titles_raw_df.head())


Rows: 8,807
Columns: 12



,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


#### Consumer Price Index (CPI) for Inflation Adjustment

The Federal Reserve Economic Data (FRED) provides historical Consumer Price Index data necessary for adjusting movie budgets and revenues to constant 2024 dollars. Without inflation adjustment, comparing a 1970s film budget to a 2020s budget is meaningless. The CPI-U (Consumer Price Index for All Urban Consumers) is the standard measure used by economists for this purpose.

Source URL: https://fred.stlouisfed.org/series/CPIAUCSL

Type: JSON (API response)

Method: API access via FRED API (programmatic)

**Dataset variables:**
- date (string) - Observation date (YYYY-MM-DD format)
- value (float) - CPI index value (1982-84 = 100 baseline)
- year (int) - Year extracted from date
- cpi (float) - Annual average CPI value

**Why this dataset:**
Inflation has been substantial over the 65-year span of this analysis (1960-2024). A \\$1 million budget in 1960 equals approximately \\$10.4 million in 2024 dollars. Without adjustment, financial analysis would be dominated by recency bias, making modern films appear far more expensive than historical films when they may actually be comparable in real terms. The FRED API provides authoritative, regularly-updated data from the U.S. Bureau of Labor Statistics.

**Significance of variables:**
- **year/cpi:** Used to calculate inflation multiplier: `adjusted_amount = original * (CPI_2024 / CPI_year)`
- **Annual averaging:** Smooths monthly volatility for more stable year-over-year comparisons

**Adjustment formula:**
```
budget_adj_2024 = budget_original × (CPI_2024 / CPI_release_year)
revenue_adj_2024 = revenue_original × (CPI_2024 / CPI_release_year)
```

In [10]:
def get_cpi_data(start_year=1960, end_year=2025, local_filename='cpi_data.json', path="data/raw", download_if_exists=False):
    """
    Get CPI data from FRED API
    CPI-U (Consumer Price Index for all Urban consumers)
    """
    
    filepath = f"{path}/{local_filename}"
    
    # Get cached data if we have it
    if os.path.exists(filepath) and download_if_exists is False:
        print(f"{filepath} already exists. If you wish to re-download it, set download_if_exists=True")
        return pd.read_json(filepath)
    
    url = 'https://api.stlouisfed.org/fred/series/observations'
    params = {
        'series_id': 'CPIAUCSL',
        'api_key': os.getenv('FRED_API'),
        'file_type': 'json',
        'observation_start': f'{start_year}-01-01',
        'observation_end': f'{end_year}-12-31',
        'frequency': 'a'
    }
    
    try:
        # Get the data as JSON
        response = httpx.get(url, params=params)
        response.raise_for_status()

        data = response.json()

        # Create dataframe
        cpi_df = pd.DataFrame(data['observations'])
        
        # Remove invalid data rows
        cpi_df = cpi_df[cpi_df['value'] != "."]
        
        # Transform data
        cpi_df['year'] = pd.to_datetime(cpi_df['date']).dt.year
        cpi_df['cpi'] = pd.to_numeric(cpi_df['value'])
        
        cpi_df = cpi_df[['year', 'cpi']]
        
        # Cache transformed data to disk
        cpi_df.to_json(filepath, orient='records')

        return cpi_df
        
    except httpx.HTTPStatusError as e:
        print(f"Error during request: {e}")
        
    except httpx.RequestError as e:
        print(f"An error occurred while requesting: {e}")

In [11]:
cpi_df = get_cpi_data()
display_shape(cpi_df)
display(cpi_df.head())
display(cpi_df['year'].unique())

data/raw/cpi_data.json already exists. If you wish to re-download it, set download_if_exists=True

Rows: 65
Columns: 2



,year,cpi
0,1960,29.585
1,1961,29.902
2,1962,30.253
3,1963,30.633
4,1964,31.038


array([1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970,
       1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981,
       1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992,
       1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
       2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024])

## **2. Assess Data**

Before cleaning and merging the datasets, we must identify data quality and tidiness issues that could impact our analysis. This assessment phase follows a structured approach: each issue is evaluated both visually (inspecting sample data) and programmatically (using pandas methods to quantify the problem), with clear justifications for why each issue matters for our research questions.

This section documents the four most critical issues—two quality problems and two tidiness violations—that must be resolved before proceeding with the streaming vs theatrical analysis. Quality issues relate to the accuracy, completeness, and validity of individual data values (missing identifiers, incorrect null encodings, zero-value anomalies). Tidiness issues concern the structural organization of data (observational units split across tables, multiple values stored in single columns).

While these four issues represent the most significant barriers to analysis, additional data quality concerns will be identified and addressed during the cleaning phase. These include filtering datasets to relevant records (movies only, excluding TV shows), standardizing column names and data types, handling delimiter inconsistencies across platforms, and selecting the subset of variables needed for our specific research questions. The comprehensive cleaning strategy will be detailed in Section 3.

**Assessment approach:**
- **Visual assessment:** Display sample data using `.head()`, `.sample()`, or filtered subsets to observe patterns
- **Programmatic assessment:** Use pandas methods (`.isna()`, `.sum()`, `.duplicated()`, `.value_counts()`) to quantify issues
- **Justification:** Explain why each issue impacts the analysis and which data quality/tidiness principle it violates

### Quality Issue 1:

In [12]:
# Visual assessment
print("Sample of movies without IMDb IDs:")
tmdb_movies_raw_df[tmdb_movies_raw_df['imdb_id'].isna()][['id', 'original_title', 'release_year']].head(10)

Sample of movies without IMDb IDs:


,id,original_title,release_year
548,355131,Sense8: Creating the World,2015
997,287663,Star Wars Rebels: Spark of Rebellion,2014
1528,15257,Hulk vs. Wolverine,2009
1750,101907,Hulk vs. Thor,2009
2401,45644,Opeth: In Live Concert At The Royal Albert Hall,2010
4797,369145,Doctor Who: The Snowmen,2012
4872,269177,Party Bercy,2012
6071,279954,Portal: Survive!,2013
7527,50127,Fallen: The Journey,2007
7809,50128,Fallen: The Destiny,2007


In [13]:
# Programmatic assessment
def check_missing_imdb_ids(df):
    missing_imdb_ids = df['imdb_id'].isna().sum()
    total_movies = len(df)
    print(f"\nMissing IMDb IDs: {missing_imdb_ids} ({missing_imdb_ids/total_movies*100:.1f}%)")
    
check_missing_imdb_ids(tmdb_movies_raw_df)


Missing IMDb IDs: 10 (0.1%)


#### Issues and Justifications

The TMDb dataset exhibits a completeness quality issue with missing IMDb identifiers: 10 movies (0.1% of the dataset) lack `imdb_id` values.

Visual assessment reveals these missing IDs occur across various years (2007-2015) and include diverse content types from animated shorts ("Sense8: Creating the World") to concert films ("Opeth: In Live Concert") to TV-movie crossovers ("Hulk vs. Wolverine", "Doctor Who: The Snowmen").

This violates the completeness principle and directly impacts the project's primary merge strategy: these 10 movies cannot be linked to IMDb's rating data via the standard tconst-to-imdb_id join, resulting in incomplete rating comparisons. While the percentage is small, it demonstrates that not all theatrical releases receive IMDb coverage, and these gaps will require alternative matching strategies (fuzzy matching on title+year) if IMDb rating data is needed for these titles.

For the streaming comparison analysis, these movies may still provide value through TMDb-only metrics, but they cannot contribute to the IMDb vs TMDb rating system comparison.

### Quality Issue 2:

In [14]:
# Visual assessment
print("Sample of IMDb data with \\N values:")
imdb_title_basics_raw_df.head(20)

Sample of IMDb data with \N values:


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,Short
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"
7,tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short"
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
9,tt0000010,short,Leaving the Factory,La sortie de l'usine Lumière à Lyon,0,1895,\N,1,"Documentary,Short"


In [15]:
# Programmatic assessment
def check_imdb_for_odd_null(df, name, columns=None):
    print(f"\nChecking for \\N null values in IMDb {name}:")
    
    if columns is None:
        columns = df.columns
    
    for col in columns:
        n_count = (df[col] == '\\N').sum()
        if n_count > 0:
            print(f"  {col}: {n_count:,} \\N values")
            
check_imdb_for_odd_null(imdb_title_basics_raw_df, "title basics")
check_imdb_for_odd_null(imdb_title_ratings_raw_df, "title ratings")


Checking for \N null values in IMDb title basics:
  startYear: 1,446,576 \N values
  endYear: 11,966,049 \N values
  runtimeMinutes: 7,801,758 \N values
  genres: 529,185 \N values

Checking for \N null values in IMDb title ratings:


#### Issues and Justifications

The IMDb datasets violate the completeness quality principle by using the string literal '\N' to represent missing values instead of proper null values. This appears throughout multiple columns including `startYear`, `endYear`, `runtimeMinutes`, and `genres`.

This encoding causes several data quality problems: (1) columns that should be numeric (like `runtimeMinutes` and `startYear`) are interpreted as object/string types due to the presence of '\N' text, preventing mathematical operations; (2) standard pandas null-handling methods like `isna()` and `fillna()` do not recognize '\N' as missing data; (3) these pseudo-nulls skew data type detection and require special handling in all analytical operations.

This is a validity issue that must be corrected by converting '\N' strings to proper pandas null values (np.nan or None) before any numeric operations, type conversions, or null-value analysis can be performed reliably. The issue affects approximately 10.5 million records in title.basics alone.

### Tidiness Issue 1:

In [16]:
# Visual assessment
print("IMDb Title Basics columns:", imdb_title_basics_raw_df.columns.tolist())
print("IMDb Ratings columns:", imdb_title_ratings_raw_df.columns.tolist())
print("\nBoth have 'tconst' - indicating they should be merged")

IMDb Title Basics columns: ['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres']
IMDb Ratings columns: ['tconst', 'averageRating', 'numVotes']

Both have 'tconst' - indicating they should be merged


In [17]:
# Programmatic assessment
print(f"\nTitle Basics has {len(imdb_title_basics_raw_df):,} records")
print(f"Title Ratings has {len(imdb_title_ratings_raw_df):,} records")
print(f"Movies with ratings: {imdb_title_ratings_raw_df['tconst'].isin(imdb_title_basics_raw_df['tconst']).sum():,}")


Title Basics has 12,116,115 records
Title Ratings has 1,607,373 records
Movies with ratings: 1,607,367


#### Issues and Justifications

The IMDb movie data violates the tidiness principle that "each type of observational unit forms a table." Movies (identified by tconst) are split across two separate tables: *title.basics* contains descriptive metadata (title, year, runtime, genres), while *title.ratings* contains audience reception data (rating, vote count). Both tables describe the same observational unit (movies), meaning information about a single movie is unnecessarily fragmented.

This structure requires joining to access complete movie information and increases the risk of data inconsistency. The tables should be merged on the `tconst` key to create a single unified dataset where each row represents one complete movie observation with all relevant attributes in a single record.

### Tidiness Issue 2: 

In [18]:
# Visual assessment
print("Sample of pipe-delimited genres in TMDb:")
display(tmdb_movies_raw_df[['original_title', 'genres']].head())

Sample of pipe-delimited genres in TMDb:


,original_title,genres
0,Jurassic World,Action|Adventure|Science Fiction|Thriller
1,Mad Max: Fury Road,Action|Adventure|Science Fiction|Thriller
2,Insurgent,Adventure|Science Fiction|Thriller
3,Star Wars: The Force Awakens,Action|Adventure|Science Fiction|Fantasy
4,Furious 7,Action|Crime|Thriller


In [19]:
# Programmatic assessment
print(f"\nGenres column contains pipe-delimited values")
print(f"Example: {tmdb_movies_raw_df['genres'].iloc[0]}")


Genres column contains pipe-delimited values
Example: Action|Adventure|Science Fiction|Thriller


#### Issues and Justifications

The `genres` column in the TMDb dataset violates the tidiness principle that "each variable forms a column." Instead of storing genre as a single atomic value, the column contains pipe-delimited strings (e.g., "Action|Adventure|Science Fiction|Thriller") with multiple genre values concatenated in a single cell.

This structure prevents straightforward filtering, aggregation, and analysis by genre. For example, identifying all "Action" movies requires string parsing rather than simple column filtering. While splitting genres into separate rows would normalize the data structure, for this analysis we will keep genres as cleaned lists to maintain one-row-per-movie structure while ensuring the values are properly separated and accessible for analysis. This approach balances analytical flexibility with data structure simplicity.

## 3. Clean Data

This section systematically addresses the four critical data quality and tidiness issues identified during assessment, validates each cleaning operation, and prepares the datasets for merging. The cleaning process follows a methodical approach:

1. **Address Quality Issues:** Remove incomplete records and standardize null value encodings
2. **Resolve Tidiness Issues:** Merge fragmented tables and transform multi-value columns
3. **Dataset-Specific Cleaning:** Filter to relevant records (movies only), standardize data types, validate year fields
4. **Variable Selection:** Remove unnecessary columns to focus on analysis-relevant features
5. **Validation:** Verify each cleaning step with programmatic or visual confirmation

All cleaning operations include explicit justifications for methodology and clear validation of successful execution. The final combined dataset includes all variables needed to answer the research question: how streaming platforms compare to theatrical releases in financial performance, audience reception, and content characteristics across six decades.

In [20]:
# Make copy of TMDb dataframe
tmdb_movies_clean_df = tmdb_movies_raw_df.copy()

# Merge IMDb basic and ratngs dataframes into a new dataframe. We're only keeping movies with ratings
imdb_title_clean_df = pd.merge(imdb_title_basics_raw_df, imdb_title_ratings_raw_df, on='tconst', how='inner')

# Make copy of all streaming platform dataframes
amazon_prime_titles_clean_df = amazon_prime_titles_raw_df.copy()
disney_plus_titles_clean_df = disney_plus_titles_raw_df.copy()
hulu_titles_clean_df = hulu_titles_raw_df.copy()
netflix_titles_clean_df = netflix_titles_raw_df.copy()

# We won't be cleaning or modifying cpi_df so no need to make a copy of it

### Quality Issue 1: Missing IMDb IDs

In [21]:
# Get TMDb rows where we have an IMDb ID
tmdb_movies_clean_df = tmdb_movies_clean_df[tmdb_movies_clean_df['imdb_id'].notna()]

In [22]:
check_missing_imdb_ids(tmdb_movies_clean_df)


Missing IMDb IDs: 0 (0.0%)


#### Issues and Justifications

We filtered to keep only movies with valid IMDb IDs because:

1. IMDb IDs are essential for merging with IMDb ratings data
2. Only 10 movies (0.1%) lack IMDb IDs - negligible data loss
3. Alternative approaches (fuzzy matching by title/year) are complex and error-prone
4. These 10 movies would be excluded anyway during the merge operation

### Quality Issue 2: IMDb data with \N values

In [23]:
imdb_title_clean_df.replace('\\N', np.nan, inplace=True)

In [24]:
check_imdb_for_odd_null(imdb_title_clean_df, "title clean")


Checking for \N null values in IMDb title clean:


#### Issues and Justifications

We replaced IMDb's '\N' notation with pandas np.nan because:

1. '\N' is IMDb's text encoding for missing data, not a proper null value
2. Pandas requires np.nan for accurate null detection (.isnull(), .notna())
3. Proper nulls enable correct statistical calculations (mean, median ignore nulls)
4. This is standard practice when working with IMDb datasets
5. Allows pandas to handle missing data appropriately during merging

### Tidiness Issue 1: Merge two IMDb tables

In [25]:
# In lieu of making a copy and then merging the copies, we merged the raw datasets into a copy previously

In [26]:
# We can visualy see the averageRating and numVotes column added and the tables merged
imdb_title_clean_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short",5.7,2187
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short",5.5,307
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892,NaN,5,"Animation,Comedy,Romance",6.4,2273
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short",5.2,196
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,Short,6.2,3012


#### Issues and Justifications

We merged the two IMDb tables (title.basics + title.ratings) because:

1. Both tables describe the same observational unit (movies/titles)
2. Having movies split across tables violates the tidiness rule: "Each type of observational unit forms a table"
3. The tconst column serves as a unique identifier linking both tables
4. Inner join ensures we only retain movies with ratings (higher quality dataset)
5. Results in a single, complete table with both metadata and audience ratings

### Tidiness Issue 2: TMDb genres column

In [27]:
# Split pipe-delimited genres into Python tuples - was going to do lists but can't use drop_duplciates() with lists

def convert_genres_to_tuple(df, sep='|', inplace=False):
    genres = df['genres'].fillna('').str.split(sep).apply(tuple)
            
    if inplace is True:
        df['genres'] = genres
    else:
        return genres

convert_genres_to_tuple(tmdb_movies_clean_df, inplace=True)

In [28]:
display(tmdb_movies_clean_df[['original_title', 'genres']].head())

,original_title,genres
0,Jurassic World,"(Action, Adventure, Science Fiction, Thriller)"
1,Mad Max: Fury Road,"(Action, Adventure, Science Fiction, Thriller)"
2,Insurgent,"(Adventure, Science Fiction, Thriller)"
3,Star Wars: The Force Awakens,"(Action, Adventure, Science Fiction, Fantasy)"
4,Furious 7,"(Action, Crime, Thriller)"


#### Issues and Justifications

We split pipe-delimited genres into Python tuples because:

1. Pipe-delimited strings violate "each variable forms a column" - multiple values encoded in single cell
2. String format prevents programmatic analysis (filtering, counting, grouping by genre)
3. Python tuples are hashable, enabling use of drop_duplicates() and other pandas operations that require immutable values
4. Tuples maintain one-row-per-movie structure while providing iterable access to individual genres
5. Easy conversion to lists if mutability needed: list(genres_tuple)
6. Avoids data duplication that would occur with exploded rows (one row per genre)

### Remove Unnecessary Variables and Combine Datasets

Before merging the TMDb and IMDb datasets, we perform dataset-specific cleaning and variable selection. This process involves:

1. **TMDb Cleaning:**
   - Verify data integrity (check for zeros, duplicates, validate years)
   - Remove non-analytical columns (homepage, keywords, production companies)
   - Keep variables essential for streaming vs theatrical comparison

<p></p>

2. **IMDb Cleaning:**
   - Filter to movies only (exclude TV shows, shorts)
   - Convert string-encoded numbers to proper numeric types
   - Select only rating-related variables (tconst, averageRating, numVotes)

<p></p>

3. **Column Selection Rationale:**
   - Financial: budget, revenue (for performance comparison)
   - Audience: vote_average, vote_count (TMDb), averageRating, numVotes (IMDb)
   - Content: genres, runtime (for characteristic analysis)
   - Temporal: release_year (for time evolution analysis)
   - Identifiers: imdb_id, original_title (for merging and validation)

The goal is a streamlined dataset containing only the variables needed to address our research questions while eliminating noise from production metadata and internal IDs.

#### TMDb Checks

In [29]:
# We're going to drop release_date from TMDb. Let's make sure we have all the release_year values first.
#
# Check for nulls in release_year
print(f"Null release_years: {tmdb_movies_clean_df['release_year'].isnull().sum()}")

# Extract year from release_date for comparison
tmdb_movies_clean_df['date_year'] = tmdb_movies_clean_df['release_date'].str.split('/').str[-1]

# Transform 2-digit year to 4-digit year
def transform_year(year_2digit):
    """
    Convert 2-digit year to 4-digit year.
    
    Assumes movies with year < 25 are from 2000s (2000-2024),
    and movies with year >= 25 are from 1900s (1925-1999).
    
    This cutoff is appropriate for a movie dataset spanning
    1960-2024, as no movies in the dataset would be from the 1920s.
    
    Args:
        year_2digit: Two-digit year as int or string
        
    Returns:
        Four-digit year as int
        
    Example:
        transform_year(15) -> 2015
        transform_year(95) -> 1995
    """
    year_2digit = int(year_2digit)
    
    if year_2digit < 25:
        return 2000 + year_2digit
    else:
        return 1900 + year_2digit

tmdb_movies_clean_df['date_year'] = tmdb_movies_clean_df['date_year'].apply(transform_year)

# Check for nulls in extracted year
print(f"Null date_years: {tmdb_movies_clean_df['date_year'].isnull().sum()}")

# Check min and max values in extracted year
print(f"\nMin date_year: {tmdb_movies_clean_df['date_year'].min()}")
print(f"Max date_year: {tmdb_movies_clean_df['date_year'].max()}")

# Check min and max values in release_year
print(f"\nMin release_year: {tmdb_movies_clean_df['release_year'].min()}")
print(f"Max release_year: {tmdb_movies_clean_df['release_year'].max()}")

# Compare release_year vs date_year
mismatches = tmdb_movies_clean_df[
    tmdb_movies_clean_df['release_year'] != tmdb_movies_clean_df['date_year']
]
print(f"\nYear mismatches: {len(mismatches)}")

# Show examples of mismatches if any
if len(mismatches) > 0:
    print("\nMismatch examples:")
    print(mismatches[['original_title', 'release_date', 'release_year', 'date_year']].head(10))

# Show distribution of differences
if len(mismatches) > 0:
    year_diff = mismatches['release_year'] - mismatches['date_year']
    print(f"\nYear difference stats:")
    print(year_diff.describe())

Null release_years: 0
Null date_years: 0

Min date_year: 1960
Max date_year: 2015

Min release_year: 1960
Max release_year: 2015

Year mismatches: 0


In [30]:
# Check for zero budgets/revenues
print(f"TMDb zero budgets: {(tmdb_movies_clean_df['budget'] == 0).sum()}")
print(f"TMDb zero revenues: {(tmdb_movies_clean_df['revenue'] == 0).sum()}")

# Convert zero budgets and revenues to NaN
tmdb_movies_clean_df['budget'] = tmdb_movies_clean_df['budget'].replace(0, np.nan)
tmdb_movies_clean_df['revenue'] = tmdb_movies_clean_df['revenue'].replace(0, np.nan)

# Validate
print(f"\nAfter conversion:")
print(f"Budget nulls: {tmdb_movies_clean_df['budget'].isna().sum():,}")
print(f"Revenue nulls: {tmdb_movies_clean_df['revenue'].isna().sum():,}")
print(f"Movies with both financial values: {tmdb_movies_clean_df[['budget', 'revenue']].notna().all(axis=1).sum():,}")

# Check for duplicate IMDb IDs
print(f"\nDuplicate imdb_ids in TMDb: {tmdb_movies_clean_df['imdb_id'].duplicated().sum()}")

# Check for zero runtimes
print(f"TMDb zero runtimes: {(tmdb_movies_clean_df['runtime'] == 0).sum()}")

TMDb zero budgets: 5687
TMDb zero revenues: 6006

After conversion:
Budget nulls: 5,687
Revenue nulls: 6,006
Movies with both financial values: 3,855

Duplicate imdb_ids in TMDb: 1
TMDb zero runtimes: 31


#### TMDb Cleaning

In [31]:
# Clean up TMDb dataset

# Drop duplicates
tmdb_movies_clean_df.drop_duplicates(inplace=True)

# Remove non-critical columns
tmdb_movies_clean_df.drop([
    'id', 'popularity', 'homepage', 'keywords', 'tagline', 'overview', 'production_companies', 
    'budget_adj', 'revenue_adj', 'cast', 'director', 'release_date', 'date_year'
], axis=1, inplace=True, errors='ignore')

#### TMDb Final View

In [32]:
print(f"\nDuplicate imdb_ids in TMDb: {tmdb_movies_clean_df['imdb_id'].duplicated().sum()}\n")
display(tmdb_movies_clean_df.head())
display(tmdb_movies_clean_df.info())


Duplicate imdb_ids in TMDb: 0



,imdb_id,budget,revenue,original_title,runtime,genres,vote_count,vote_average,release_year
0,tt0369610,150000000.0,1.513529e+09,Jurassic World,124,"(Action, Adventure, Science Fiction, Thriller)",5562,6.5,2015
1,tt1392190,150000000.0,3.784364e+08,Mad Max: Fury Road,120,"(Action, Adventure, Science Fiction, Thriller)",6185,7.1,2015
2,tt2908446,110000000.0,2.952382e+08,Insurgent,119,"(Adventure, Science Fiction, Thriller)",2480,6.3,2015
3,tt2488496,200000000.0,2.068178e+09,Star Wars: The Force Awakens,136,"(Action, Adventure, Science Fiction, Fantasy)",5292,7.5,2015
4,tt2820852,190000000.0,1.506249e+09,Furious 7,137,"(Action, Crime, Thriller)",2947,7.3,2015


<class 'pandas.core.frame.DataFrame'>
Index: 10855 entries, 0 to 10865
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   imdb_id         10855 non-null  object 
 1   budget          5168 non-null   float64
 2   revenue         4849 non-null   float64
 3   original_title  10855 non-null  object 
 4   runtime         10855 non-null  int64  
 5   genres          10855 non-null  object 
 6   vote_count      10855 non-null  int64  
 7   vote_average    10855 non-null  float64
 8   release_year    10855 non-null  int64  
dtypes: float64(3), int64(3), object(3)
memory usage: 848.0+ KB


None

#### IMDb Checks

In [33]:
# Defined here but used later during cleanup
drop_columns = ['isAdult']

print(f"Duplicate tconsts in IMDb: {imdb_title_clean_df['tconst'].duplicated().sum()}")
print(f"IMDb zero runtimes: {(imdb_title_clean_df['runtimeMinutes'] == 0).sum()}")

display_shape(imdb_title_clean_df)

# Check the situation on null values in columns
def check_null_column_values(df):
    print("Columns with NULL values:")
    for col in df.columns:
        n_count = (df[col].isnull()).sum()
        if n_count > 0:
            print(f"  {col}: {n_count:,} null values")
    print()

check_null_column_values(imdb_title_clean_df)
    
# Nearly all endYear values are null (1,538,909 / 1,607,367); drop column
drop_columns.append('endYear')

# Fill null numeric columns with zero
# Technically clean-up but we need to do this before the next check
imdb_title_clean_df['startYear'].fillna(0, inplace=True)
imdb_title_clean_df['endYear'].fillna(0, inplace=True)
imdb_title_clean_df['runtimeMinutes'].fillna(0, inplace=True)

check_null_column_values(imdb_title_clean_df)

# Check runtimeMinutes for non-numeric values
non_numeric_mask = pd.to_numeric(imdb_title_clean_df['runtimeMinutes'], errors='coerce').isnull()
non_numeric_rows = imdb_title_clean_df[non_numeric_mask]
non_numeric_row_count = non_numeric_rows.shape[0]

print(f"Found {non_numeric_row_count:,} rows where runtimeMinutes is not a numeric value:\n")
display(imdb_title_clean_df[non_numeric_mask])

Duplicate tconsts in IMDb: 0
IMDb zero runtimes: 0

Rows: 1,607,367
Columns: 11

Columns with NULL values:
  primaryTitle: 1 null values
  originalTitle: 1 null values
  startYear: 255 null values
  endYear: 1,538,909 null values
  runtimeMinutes: 472,522 null values
  genres: 22,361 null values

Columns with NULL values:
  primaryTitle: 1 null values
  originalTitle: 1 null values
  genres: 22,361 null values

Found 4 rows where runtimeMinutes is not a numeric value:



,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
594683,tt12149332,tvEpisode,Jeopardy! College Championship Semifinal Game ...,0,2020,0,45,"Family,Game-Show",NaN,6.9,9
1049604,tt27404292,tvEpisode,Nord-Koreas röda prinsessa\tNord-Koreas röda p...,0,2022,0,0,Documentary,NaN,7.4,5
1080033,tt28535095,tvEpisode,I'm into Sharif Boys\tI'm into Sharif Boys,0,2023,0,0,Reality-TV,NaN,2.8,46
1283807,tt3984412,tvEpisode,"I'm Not Going to Come Last, I'm Just Going to ...",0,2014,0,0,"Game-Show,Reality-TV",NaN,7.8,7


#### IMDb Cleanup

In [34]:
# There are only a few non-numeric and none of them are movies.
# Also, these rows are missing a bunch of data we want. Drop them.
imdb_title_clean_df = imdb_title_clean_df[~non_numeric_mask]

# Convert columns
imdb_title_clean_df['startYear'] = pd.to_numeric(imdb_title_clean_df['startYear'])
imdb_title_clean_df['endYear'] = pd.to_numeric(imdb_title_clean_df['endYear'])
imdb_title_clean_df['runtimeMinutes'] = pd.to_numeric(imdb_title_clean_df['runtimeMinutes'])

# Remove non-critical columns
imdb_title_clean_df.drop(drop_columns, axis=1, inplace=True, errors='ignore')

# Convert genres to tuples to match TMBd format
convert_genres_to_tuple(imdb_title_clean_df, sep=",", inplace=True)

#### IMDb Final View

In [35]:
# Final look at IMDb dataset
display(imdb_title_clean_df.head())
display(imdb_title_clean_df.info())

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,1894,1,"(Documentary, Short)",5.7,2187
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,1892,5,"(Animation, Short)",5.5,307
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,1892,5,"(Animation, Comedy, Romance)",6.4,2273
3,tt0000004,short,Un bon bock,Un bon bock,1892,12,"(Animation, Short)",5.2,196
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,1893,1,"(Short,)",6.2,3012


<class 'pandas.core.frame.DataFrame'>
Index: 1607363 entries, 0 to 1607366
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   tconst          1607363 non-null  object 
 1   titleType       1607363 non-null  object 
 2   primaryTitle    1607362 non-null  object 
 3   originalTitle   1607362 non-null  object 
 4   startYear       1607363 non-null  int64  
 5   runtimeMinutes  1607363 non-null  int64  
 6   genres          1607363 non-null  object 
 7   averageRating   1607363 non-null  float64
 8   numVotes        1607363 non-null  int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 122.6+ MB


None

#### TMDb and IMDb Summary

In [36]:
# ============================================================================
# Final Validation Summary
# ============================================================================

print("=" * 70)
print("DATA CLEANING COMPLETE - SUMMARY")
print("=" * 70)

print("\nTMDB DATASET:")
print(f"   Rows: {len(tmdb_movies_clean_df):,}")
print(f"   Columns: {len(tmdb_movies_clean_df.columns)}")
print(f"   Column names: {', '.join(tmdb_movies_clean_df.columns.tolist())}")

print("\nIMDB DATASET:")
print(f"   Rows: {len(imdb_title_clean_df):,}")
print(f"   Columns: {len(imdb_title_clean_df.columns)}")
print(f"   Column names: {', '.join(imdb_title_clean_df.columns.tolist())}")

DATA CLEANING COMPLETE - SUMMARY

TMDB DATASET:
   Rows: 10,855
   Columns: 9
   Column names: imdb_id, budget, revenue, original_title, runtime, genres, vote_count, vote_average, release_year

IMDB DATASET:
   Rows: 1,607,363
   Columns: 9
   Column names: tconst, titleType, primaryTitle, originalTitle, startYear, runtimeMinutes, genres, averageRating, numVotes


#### TMDb and IMDb Merging

In [37]:
## TODO:
# Do IMDb runtimes fill in any gaps for nan runtimes in TMDb or vice versa? 
# Do runtimes agree between TMDb and IMDb? If not, what's the difference? If small, maybe use mean.
# Do TMDb and IMDb genres agree? If not, what do we do? Merge?


# After merge, compare runtimes
#merged_df['runtime_diff'] = abs(merged_df['runtime'] - merged_df['runtimeMinutes'])
#print(merged_df['runtime_diff'].describe())

# If differences are small (median <5 minutes), use mean
# If large discrepancies, investigate and document decision

# Runtime Analysis - Check for zeros and missing values
print("RUNTIME CHECKS:\n")

# TMDb runtime analysis
tmdb_zero_runtime = (tmdb_movies_clean_df['runtime'] == 0).sum()
tmdb_null_runtime = tmdb_movies_clean_df['runtime'].isna().sum()
print(f"TMDb - Zero runtimes: {tmdb_zero_runtime:,}")
print(f"TMDb - Null runtimes: {tmdb_null_runtime:,}")
print(f"TMDb - Runtime range: {tmdb_movies_clean_df['runtime'].min()} - {tmdb_movies_clean_df['runtime'].max()}")

# IMDb runtime analysis
imdb_zero_runtime = (imdb_title_clean_df['runtimeMinutes'] == 0).sum()
imdb_null_runtime = imdb_title_clean_df['runtimeMinutes'].isna().sum()
print(f"\nIMDb - Zero runtimes: {imdb_zero_runtime:,}")
print(f"IMDb - Null runtimes: {imdb_null_runtime:,}")
print(f"IMDb - Runtime range: {imdb_title_clean_df['runtimeMinutes'].min()} - {imdb_title_clean_df['runtimeMinutes'].max()}")

RUNTIME CHECKS:

TMDb - Zero runtimes: 31
TMDb - Null runtimes: 0
TMDb - Runtime range: 0 - 900

IMDb - Zero runtimes: 472,525
IMDb - Null runtimes: 0
IMDb - Runtime range: 0 - 3692080


**Runtime Handling Strategy**

Based on the analysis above:

**Zero Runtimes:**
- Zero-minute runtimes are invalid for theatrical movies (even shorts have measurable runtime)
- These represent missing data incorrectly encoded as zeros
- **Decision:** Convert zeros to NaN for accurate statistical calculations

**Missing Runtime Strategy:**
After merging TMDb and IMDb, we will:
1. Use TMDb runtime as primary (more specific to theatrical releases)
2. Fill TMDb nulls with IMDb runtimeMinutes where available
3. Document remaining nulls for potential exclusion from runtime-based analyses

In [38]:
# Convert zero runtimes to NaN in TMDb
tmdb_movies_clean_df['runtime'] = tmdb_movies_clean_df['runtime'].replace(0, np.nan)

# Validate the change
print(f"\nAfter conversion:")
print(f"TMDb - Null runtimes: {tmdb_movies_clean_df['runtime'].isna().sum():,}")


After conversion:
TMDb - Null runtimes: 31


**Genre Handling Strategy**

TMDb and IMDb use different but complementary genre systems:

**TMDb Genres:** More granular (e.g., "Science Fiction", "Adventure")
**IMDb Genres:** Includes additional categories (e.g., "Biography", "Film-Noir", "Musical")

**Key Differences**

- IMDb uses "Sci-Fi" vs TMDb's "Science Fiction"
- IMDb includes TV-specific genres (Game-Show, Reality-TV, Talk-Show, News)
- IMDb separates "Music" and "Musical"
- IMDb has "Biography" and "Film-Noir" which TMDb doesn't distinguish

**Merging Decision**

We'll merge genres from both sources to create a comprehensive genre classification:

1. **Normalize:** Map "Sci-Fi" to "Science Fiction" for consistency
2. **Filter:** Exclude TV-specific genres (Game-Show, News, Reality-TV, Talk-Show)
3. **Exclude:** Remove "Short" (it's a title type, not a genre)
4. **Preserve:** Keep valuable IMDb genres (Biography, Film-Noir, Musical, Sport)
5. **Deduplicate:** Combine into sorted tuple of unique genres

This approach maximizes genre coverage while maintaining data quality and consistency.

**Result:** Each movie has a comprehensive, normalized set of genres drawing from both authoritative sources, enabling robust content characteristic analysis.

In [39]:
# Quick check: What are the unique genres in each dataset?
def check_genres(df, name):
    # Extract unique genres from tuples
    unique_genres = set()
    for genre_tuple in df['genres'].dropna():
        # Skip empty tuples
        if genre_tuple:
            unique_genres.update(genre_tuple)

    print(f"{name} unique genres ({len(unique_genres)}):")
    print(sorted(unique_genres))
    
check_genres(tmdb_movies_clean_df, "TMDb")
print()

check_genres(imdb_title_clean_df, "IMDb")
print()

TMDb unique genres (21):
['', 'Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western']

IMDb unique genres (29):
['', 'Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'Game-Show', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']



## 4. Update your data store
Update your local database/data store with the cleaned data, following best practices for storing your cleaned data:

- Must maintain different instances / versions of data (raw and cleaned data)
- Must name the dataset files informatively
- Ensure both the raw and cleaned data is saved to your database/data store

In [ ]:
#FILL IN - saving data

## 5. Answer the research question

### **5.1:** Define and answer the research question 
Going back to the problem statement in step 1, use the cleaned data to answer the question you raised. Produce **at least** two visualizations using the cleaned data and explain how they help you answer the question.

*Research question:* FILL IN from answer to Step 1

In [ ]:
#Visual 1 - FILL IN

*Answer to research question:* FILL IN

In [ ]:
#Visual 2 - FILL IN

*Answer to research question:* FILL IN

### **5.2:** Reflection
In 2-4 sentences, if you had more time to complete the project, what actions would you take? For example, which data quality and structural issues would you look into further, and what research questions would you further explore?

*Answer:* FILL IN